# Strands Agents: Simplify AI Agents Development with Model-driven Approach

Welcome to this tutorial on Strands Agents! Strands is a powerful, open-source Python SDK from AWS that simplifies the creation of AI agents. It's designed to be model-agnostic, allowing you to use various Large Language Models (LLMs) like those from **Amazon Bedrock**, OpenAI, Anthropic, and even local models.

In this notebook, we will cover the fundamental concepts of Strands Agents and build a simple yet functional agent that can use tools to perform tasks.

**What you will learn:**

* **Core Concepts:** Understand the key components of a Strands Agent: `Agent`, `Model`, and `Tools`.
* **Basic Agent Creation:** Create a simple agent and interact with it using Amazon Bedrock.
* **Using Tools:** Equip your agent with built-in and custom tools to extend its capabilities.
* **Environment Variables:** Securely manage your AWS credentials using a `.env` file.
* **A Practical Example:** Build a "Research Assistant" agent that can search the web.

Let's get started! 🚀

In [1]:
# First, let's install the necessary packages.
# - strands-agents: The core SDK.
# - strands-agents-tools: A collection of pre-built tools.
# - python-dotenv: To load our API keys from a .env file.
# - boto3: The AWS SDK for Python, required for Amazon Bedrock.

#%pip install strands-agents strands-agents-tools python-dotenv boto3 -q

## Setting Up AWS Credentials

To use Amazon Bedrock models, you'll need AWS credentials. It's a best practice to keep these credentials out of your code. We'll use a `.env` file to store them securely.

1.  **Create a file named `.env`** in the same directory as this notebook.
2.  **Add your AWS credentials** to the `.env` file in the following format:

    ```
    AWS_ACCESS_KEY_ID="your_aws_access_key_id"
    AWS_SECRET_ACCESS_KEY="your_aws_secret_access_key"
    AWS_DEFAULT_REGION="us-west-2"  # or your preferred region
    ```

For this tutorial, we will be using Amazon Bedrock with Claude 3 Haiku. You can get AWS credentials from the [AWS IAM Console](https://console.aws.amazon.com/iam/).


In [7]:
import os
from dotenv import load_dotenv

# Load the environment variables from the .env file
load_dotenv()

# We'll check if the AWS credentials are loaded.
if "AWS_ACCESS_KEY_ID" in os.environ and "AWS_SECRET_ACCESS_KEY" in os.environ:
    print("✅ AWS credentials loaded successfully!")
else:
    print("⚠️ AWS credentials not f ound. Please check your .env file.")

✅ AWS credentials loaded successfully!


## Core Concepts: The Agent, Model, and Tools

A Strands Agent is composed of three main parts:

1.  **Model:** The LLM that powers the agent's reasoning and decision-making. Strands is model-agnostic, so you can easily switch between models from different providers.
2.  **Tools:** Functions that the agent can use to interact with the outside world, like searching the web, using a calculator, or accessing a database.
3.  **Prompt:** A set of instructions that tells the agent what to do.

Let's create our first agent. We'll start with a simple agent that doesn't have any tools yet. We will use the `BedrockModel` and specify a Claude model from Amazon Bedrock.


In [ ]:
from strands import Agent
from strands.models.bedrock import BedrockModel
import os

# 1. Define the model
# We are using the BedrockModel with Claude 3 Sonnet.
# You must have model access enabled for this model ID in your AWS account's Bedrock console.
model = BedrockModel(
    model_id="anthropic.claude-3-5-haiku-20241022-v1:0",
    region_name=os.environ.get("AWS_DEFAULT_REGION", "us-west-2") # Uses the region from .env or defaults to us-west-2
)

# 2. Define the agent
# We'll give it a system prompt to define its personality.
agent = Agent(
    model=model,
    system_prompt="You are a friendly and helpful assistant."
)

# 3. Interact with the agent
response = agent("Hello! What can you do?")

Hi there! I'm Claude, an AI created by Anthropic to be helpful, honest, and harmless. I can help with all sorts of tasks like:

- Writing and editing
- Analysis and research
- Math and coding
- Answering questions
- Creative brainstorming
- Problem-solving

I aim to be direct, substantive, and ethical in my responses. I won't help with anything harmful, and I'll be upfront if I'm unsure about something. What would you like assistance with today?

## Extending Capabilities with Tools

Our agent is not very useful yet because it can't do anything other than talk. Let's give it some tools! Strands provides a variety of pre-built tools in the `strands-agents-tools` package. We'll give our agent a `calculator` and the ability to get the `current_time`.

You can also create your own custom tools by simply decorating a Python function with `@tool`. Let's see how this works in practice.


In [10]:
from strands import tool
from strands_tools import calculator, current_time
import datetime

# Let's create a custom tool to say hello.
@tool
def say_hello(name: str) -> str:
    """A tool to say hello to a person."""
    return f"Hello, {name}!"

# Create a new agent with the built-in and custom tools.
agent_with_tools = Agent(
    model=model,
    system_prompt="You are a helpful assistant that can use tools.",
    tools=[calculator, current_time, say_hello]
)

# Now, let's ask the agent a question that requires a tool.
response_calculator = agent_with_tools("What is 25 * 8?")

print("-" * 20)

# And another one that uses our custom tool.
response_hello = agent_with_tools("Say hello to the world.")

print("-" * 20)

# And one that uses the current_time tool.
response_time = agent_with_tools("What time is it?")

I'll help you calculate that using the calculator tool.
Tool #1: calculator


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 25 * 8              │                                                                            │
│  │ Result    │ 200                 │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

The result of 25 * 8 is 200.--------------------
I'll use the say_hello tool to greet the world.
Tool #2: say_hello
There you go! A friendly hello to the world.--------------------
I'll retrieve the current time using the current_time tool. Since no specific timezone was mentioned, it will default to UTC.
Tool #3: current_time
The current time in UTC is 2025-08-20T15:16:46.884773+00:00. This is in ISO 8601 format, which includes the date, time, and timezone offset.

Would you like me to show the time in a specific timezone?

## A Practical Example: The Research Assistant

Now, let's build a more practical agent. We'll create a "Research Assistant" that can search the web to answer questions. For this, we'll use an `http_request` tool which allows the agent to make HTTP requests to fetch information from the web.

We will also see how the agent can use multiple tools in a sequence to answer a more complex question that requires both research and calculation.


In [15]:
from strands_tools import http_request

# Create our Research Assistant agent
research_assistant = Agent(
    model=model,
    system_prompt="You are a research assistant. Your job is to use the web search tool to answer questions accurately.",
    tools=[http_request, calculator] # Give it the web search and calculator tools
)

# Let's ask a research question.
research_response = research_assistant("What is the latest news about Strands Agents?")

print("-" * 20)

# A more complex question that might require search and calculation.
complex_response = research_assistant("If a product costs $199 and is on a 15% discount, what is the final price?")

Let me search for the latest information about Strands Agents.
Tool #1: http_request


╭───────────────────────────────────── 🚀 HTTP Request Preview: GET /search ──────────────────────────────────────╮
│                                                                                                                 │
│   Method    GET                                                                                                 │
│   URL       https://www.google.com/search?q=Strands+Agents+latest+news                                          │
│   Headers   {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)    │
│             Chrome/91.0.4472.124 Safari/537.36'}                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Sending request...

╭───────────────────────────────────────────── ❌ HTTP Response: 0  ──────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│     Status         429 Too Many Requests                                                                        │
│     URL            https://www.google.com/sorry/index?continue=https://www.google.com/search%3Fq%3DStrands…     │
│     Content-Type   text/html                                                                                    │
│     Size           3,217 bytes (3.1 KB)                                                                         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────── Redirect Information ───────╮
│                                    │
│ Followed 1 redirect(s): 302 -> 429 │
│                                    │
╰────────────────────────────────────╯

...

                                                 Response Headers                                                  
╭──────────────────────────────┬──────────────────────────────────────────────────────────────────────────────────╮
│ Header                       │ Value                                                                            │
├──────────────────────────────┼──────────────────────────────────────────────────────────────────────────────────┤
│ Content-Type                 │ text/html; charset=utf-8                                                         │
│ Vary                         │ Sec-Fetch-Dest, Sec-Fetch-Mode, Sec-Fetch-Site                                   │
│ x-ua-compatible              │ IE=edge                                                                          │
│ Cache-Control                │ no-cache, no-store, max-age=0, must-revalidate                                   │
│ Pragma                       │ no-cache                                                                         │
│ Expires                      │ Mon, 01 Jan 1990 00:00:00 GMT                                                    │
│ Date                         │ Wed, 20 Aug 2025 15:20:47 GMT                                                    │
│ P3P                          │ CP="This is not a P3P policy! See g.co/p3phelp for more info."                   │
│ Strict-Transport-Security    │ max-age=31536000                                                                 │
│ Permissions-Policy           │ ch-ua-arch=*, ch-ua-bitness=*, ch-ua-full-version=*, ch-ua-full-version-list=*,  │
│                              │ ch-ua-model=*, ch-ua...                                                          │
│ Content-Security-Policy      │ require-trusted-types-for 'script';report-uri /_/DotsSplashUi/cspreport,         │
│                              │ script-src 'report-sample' ...                                                   │
│ Cross-Origin-Resource-Policy │ same-site                                                                        │
│ Cross-Origin-Opener-Policy   │ same-origin-allow-popups                                                         │
│ Accept-CH                    │ Sec-CH-UA-Arch, Sec-CH-UA-Bitness, Sec-CH-UA-Full-Version,                       │
│                              │ Sec-CH-UA-Full-Version-List, Sec-CH-UA-Mo...                                     │
│ reporting-endpoints          │ default="/_/DotsSplashUi/web-reports?context=eJzjCtDikmJw05Bi-LxjBmvrzXOsU4HYUO… │
│ Content-Encoding             │ gzip                                                                             │
│ Server                       │ ESF                                                                              │
│ X-XSS-Protection             │ 0                                                                                │
│ X-Frame-Options              │ SAMEORIGIN                                                                       │
│ X-Content-Type-Options       │ nosniff                                                                          │
│ Set-Cookie                   │ NID=525=NSo5kQFb7PWES_-_tdnOE2IPqhzD7gSB80f96ntDH5Rhqe9zzlMw-2QBpZ4QAQUj0nm0krj… │
│ Alt-Svc                      │ h3=":443"; ma=2592000,h3-29=":443"; ma=2592000                                   │
│ Transfer-Encoding            │ chunked                                                                          │
╰──────────────────────────────┴──────────────────────────────────────────────────────────────────────────────────╯

bedrock threw context window overflow error


I apologize, but I'm unable to retrieve current news about Strands Agents through web search at the moment. This could be due to several reasons:

1. Strands Agents might be a relatively niche or specialized topic.
2. There might be recent changes or restrictions in web searching.
3. The search terms might be too specific or not yielding recent results.

To provide you with accurate information, I recommend:
- Checking the official Strands Agents website
- Looking for recent press releases or company announcements
- Consulting industry-specific news sources or forums
- Directly contacting the company for the most up-to-date information

If you have more context about Strands Agents (such as what industry they're in, what type of company or service they provide), I can help you refine the search strategy or suggest specific sources to check for the latest news.--------------------
I'll help you calculate the final price using the calculator.
Tool #3: calculator


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 199 * (1 - 0.15)    │                                                                            │
│  │ Result    │ 169.1500000004      │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Let me break down the calculation:
- Original price: $199
- Discount percentage: 15%
- Discount amount: $199 * 0.15 = $29.85
- Final price: $199 - $29.85 = $169.15

So the final price after the 15% discount is $169.15.

## Congratulations!

You've successfully built your first Strands Agents using Amazon Bedrock!

In this tutorial, you learned how to:

* Create a Strands Agent with a specified **Bedrock model**.
* Use built-in tools and create your own custom tools.
* Manage **AWS credentials** securely with a `.env` file.
* Build a practical "Research Assistant" agent.

This is just the beginning of what you can do with Strands Agents. You can explore more advanced concepts like multi-agent orchestration, state management, and more in the official documentation.

Happy building! 🚀